# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 15 2023 2:36PM,259392,10,Enova-11514,Emerginnova,Released
1,Mar 15 2023 2:33PM,259391,10,HHHH9127,Hush Hush,Released
2,Mar 15 2023 1:35PM,259389,10,Stone-9269543,Acute Outpatient Solutions,Released
3,Mar 15 2023 12:50PM,259381,10,HH-40632,Hush Hush,Released
4,Mar 15 2023 12:50PM,259381,10,HH-40633,Hush Hush,Released
5,Mar 15 2023 12:50PM,259381,10,HH-40634,Hush Hush,Released
6,Mar 15 2023 12:50PM,259381,10,HH-40635,Hush Hush,Released
7,Mar 15 2023 12:14PM,259379,18,SEQ-9268441,Seqirus,Released
8,Mar 15 2023 12:08PM,259378,10,HHHH9126,Hush Hush,Released
9,Mar 15 2023 9:51AM,259360,19,ADV80013740,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,259379,Released,1
9,259381,Released,4
10,259389,Released,1
11,259391,Released,1
12,259392,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259379,NaN,NaN,1.0
259381,NaN,NaN,4.0
259389,NaN,NaN,1.0
259391,NaN,NaN,1.0
259392,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259291,3.0,0.0,3.0
259312,0.0,1.0,0.0
259319,0.0,1.0,0.0
259360,2.0,17.0,3.0
259378,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259291,3,0,3
259312,0,1,0
259319,0,1,0
259360,2,17,3
259378,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259291,3,0,3
1,259312,0,1,0
2,259319,0,1,0
3,259360,2,17,3
4,259378,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259291,3,,3
1,259312,,1,
2,259319,,1,
3,259360,2,17,3
4,259378,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 15 2023 2:36PM,259392,10,Emerginnova
1,Mar 15 2023 2:33PM,259391,10,Hush Hush
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions
3,Mar 15 2023 12:50PM,259381,10,Hush Hush
7,Mar 15 2023 12:14PM,259379,18,Seqirus
8,Mar 15 2023 12:08PM,259378,10,Hush Hush
9,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd"
31,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc."
37,Mar 15 2023 8:30AM,259319,10,Hush Hush
38,Mar 14 2023 3:10PM,259312,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 15 2023 2:36PM,259392,10,Emerginnova,,,1
1,Mar 15 2023 2:33PM,259391,10,Hush Hush,,,1
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions,,,1
3,Mar 15 2023 12:50PM,259381,10,Hush Hush,,,4
4,Mar 15 2023 12:14PM,259379,18,Seqirus,,,1
5,Mar 15 2023 12:08PM,259378,10,Hush Hush,,,1
6,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",2,17,3
7,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",3,,3
8,Mar 15 2023 8:30AM,259319,10,Hush Hush,,1,
9,Mar 14 2023 3:10PM,259312,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 15 2023 2:36PM,259392,10,Emerginnova,1,,
1,Mar 15 2023 2:33PM,259391,10,Hush Hush,1,,
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions,1,,
3,Mar 15 2023 12:50PM,259381,10,Hush Hush,4,,
4,Mar 15 2023 12:14PM,259379,18,Seqirus,1,,
5,Mar 15 2023 12:08PM,259378,10,Hush Hush,1,,
6,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",3,17,2
7,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",3,,3
8,Mar 15 2023 8:30AM,259319,10,Hush Hush,,1,
9,Mar 14 2023 3:10PM,259312,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 15 2023 2:36PM,259392,10,Emerginnova,1,,
1,Mar 15 2023 2:33PM,259391,10,Hush Hush,1,,
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions,1,,
3,Mar 15 2023 12:50PM,259381,10,Hush Hush,4,,
4,Mar 15 2023 12:14PM,259379,18,Seqirus,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 15 2023 2:36PM,259392,10,Emerginnova,1.0,NaN,NaN
1,Mar 15 2023 2:33PM,259391,10,Hush Hush,1.0,NaN,NaN
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions,1.0,NaN,NaN
3,Mar 15 2023 12:50PM,259381,10,Hush Hush,4.0,NaN,NaN
4,Mar 15 2023 12:14PM,259379,18,Seqirus,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 15 2023 2:36PM,259392,10,Emerginnova,1.0,0.0,0.0
1,Mar 15 2023 2:33PM,259391,10,Hush Hush,1.0,0.0,0.0
2,Mar 15 2023 1:35PM,259389,10,Acute Outpatient Solutions,1.0,0.0,0.0
3,Mar 15 2023 12:50PM,259381,10,Hush Hush,4.0,0.0,0.0
4,Mar 15 2023 12:14PM,259379,18,Seqirus,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1556250,8.0,1.0,0.0
12,259312,0.0,1.0,0.0
15,259291,3.0,0.0,3.0
18,259379,1.0,0.0,0.0
19,259360,3.0,17.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1556250,8.0,1.0,0.0
1,12,259312,0.0,1.0,0.0
2,15,259291,3.0,0.0,3.0
3,18,259379,1.0,0.0,0.0
4,19,259360,3.0,17.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,1.0,0.0
1,12,0.0,1.0,0.0
2,15,3.0,0.0,3.0
3,18,1.0,0.0,0.0
4,19,3.0,17.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,0.0
2,15,Released,3.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19
Status,,,,,
Completed,0.0,0.0,3.0,0.0,2.0
Executing,1.0,1.0,0.0,0.0,17.0
Released,8.0,0.0,3.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19
0,Completed,0.0,0.0,3.0,0.0,2.0
1,Executing,1.0,1.0,0.0,0.0,17.0
2,Released,8.0,0.0,3.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19
0,Completed,0.0,0.0,3.0,0.0,2.0
1,Executing,1.0,1.0,0.0,0.0,17.0
2,Released,8.0,0.0,3.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()